## Inpsecting the annotations.jsonl file

In [1]:
import sklearn
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import scipy.stats
import eli5
import json 
#
with open("../annotations.jsonl") as jsonl_file:
    lines = jsonl_file.readlines()
annot = [json.loads(line) for line in lines]
print("instances:\n{}".format(len(annot)))
keys = [key for key in annot[0].keys()]
print("\nall keys:\n{}".format(keys))
key_keys = ["text", "spans", "tokens"]
print("\nimportant keys:\n{}".format(key_keys))
print("\nexample text:\n{}".format(annot[0]["text"]))
n_examples = 5
print("\n{} example spans:".format(n_examples))
for span in annot[0]["spans"][:n_examples]:
    print("{}".format(span))
print("\n{} example tokens:".format(n_examples))
for token in annot[0]["tokens"][:n_examples]:
    print("{}".format(token))

instances:
140

all keys:
['text', 'meta', '_input_hash', '_task_hash', 'spans', 'tokens', '_view_id', 'answer', '_timestamp']

important keys:
['text', 'spans', 'tokens']

example text:
DORNBIRN In der Schulgasse in Dornbirn hat eine 71,93 Quadratmeter große Wohnung für einen Quadratmeterpreis von 5533,71 Euro den Besitzer gewechselt. Dieser beinhaltet auch einen Pkw-Abstellplatz. Käufer der Wohnung mit 9,86 Quadratmetern Terrasse ist die ValLiLean Beteiligungs- und Immobilienverwaltungs GmbH. Beim Verkäufer handelt es sich um die Karrenblick Projekt GmbH.  Der Kaufpreis liegt bei 398.040 Euro. Unterzeichnet wurde der Kaufvertrag am 18. September. Die Verbücherung datiert mit Oktober 2020.

5 example spans:
{'text': 'DORNBIRN', 'start': 0, 'end': 8, 'pattern': 2069086582, 'token_start': 0, 'token_end': 0, 'label': 'ORT'}
{'start': 16, 'end': 26, 'token_start': 3, 'token_end': 3, 'label': 'STRASSE'}
{'text': 'Dornbirn', 'start': 30, 'end': 38, 'pattern': 2069086582, 'token_start': 5, '

In [2]:
# def entityAndLabel_j_in_text_i(i, j, annots, verbose=False):
#     instance = annots[i]
#     text = instance["text"]
#     span = instance["spans"][j]
#     entity = text[span["start"]:span["end"]]
#     label = span["label"]
#     if verbose:
#         print("text:\n{}\n\nspan:\n{}\n\nentity:\n{}\n\nlabel:\n{}\n".format(text, span, entity, label))
#     return entity, label
#
# try:
#     print(entityAndLabel_j_in_text_i(50, 10, annot))
# except:
#     IndexError

('Juni 2020.', 'DATUM_VERBUECHERUNG')


In [3]:
def getLabel(dictList, idx):
    result = "O"
    for dict_i in dictList:
        idx_0, idx_1 = dict_i['start'], dict_i['end']
        if (idx_0<=idx) and (idx<=idx_1):
            result = dict_i["label"]
    return result 

myDictList = [
    {'start':0, 'end':3, 'label': 'ORT'},
    {'start':5, 'end':8, 'label': 'ORT2'},
    {'start':10, 'end':13, 'label': 'ORT3'}
]

for i in range(16):
    pos = i-1
    print("label for position {}:\t{}".format(pos, getLabel(myDictList, i-1)))

label for position -1:	O
label for position 0:	ORT
label for position 1:	ORT
label for position 2:	ORT
label for position 3:	ORT
label for position 4:	O
label for position 5:	ORT2
label for position 6:	ORT2
label for position 7:	ORT2
label for position 8:	ORT2
label for position 9:	O
label for position 10:	ORT3
label for position 11:	ORT3
label for position 12:	ORT3
label for position 13:	ORT3
label for position 14:	O


In [4]:
check = True
for j in range(len(annot)): # loop over instances
    a = annot[j]            # instance j
    spans = a['spans']      # list of annotation dicts
    toks = a['tokens']      # list of token dicts
    for i in range(len(toks)):                                 # loop over token dicts
        toks[i]['label'] = getLabel(spans, toks[i]['start'])   # assign label from span (if exists, otherwise "O")
        if toks[i]['label'] != "O":                            # if the token represents an entity ...
            if i==0:
                toks[i]['label'] = "B-"+toks[i]['label']       # ... and is the first in the text => "B-" + label
            else:                                              # not first token in text:
                if (toks[i]['label']==toks[i-1]['label'][2:]):
                    toks[i]['label'] = "I-"+toks[i]['label']   # > but same label as previous token => "I-" + label
                else:
                    toks[i]['label'] = "B-"+toks[i]['label']   # > but first token of an entity => "B-" + label
    annot[j]['tokens'] = toks

words_n = 3
for i in range(2):
    print("Token dictionaries for the last {} words of instance {}".format(words_n, i))
    ann = annot[i]
    for tok in ann["tokens"][:words_n]:
        print(tok)
# O => trivial class (no entity)
# B => Entity or leading token of an entity
# I => subsequent token of an entity

Token dictionaries for the last 3 words of instance 0
{'text': 'DORNBIRN', 'start': 0, 'end': 8, 'id': 0, 'ws': True, 'label': 'B-ORT'}
{'text': 'In', 'start': 9, 'end': 11, 'id': 1, 'ws': True, 'label': 'O'}
{'text': 'der', 'start': 12, 'end': 15, 'id': 2, 'ws': True, 'label': 'O'}
Token dictionaries for the last 3 words of instance 1
{'text': 'FELDKIRCH', 'start': 0, 'end': 9, 'id': 0, 'ws': True, 'label': 'B-ORT'}
{'text': 'Im', 'start': 10, 'end': 12, 'id': 1, 'ws': True, 'label': 'O'}
{'text': 'Altenreuteweg', 'start': 13, 'end': 26, 'id': 2, 'ws': True, 'label': 'B-STRASSE'}


In [5]:
# ToDo
# > undertand the main stuff below
# > write a function to list tokens and labels next to each other
# > when done with everything, do the following
# >> document all this nicely
# >> acknowledge Vasco
# >> point out reading list and links

In [6]:
# now convert annotation tokens into list (sentences) of lists (tokens) format for sklearn_crfsuite.CRF
train_sents=[] 
for j in range(0,len(annot)):
    a = annot[j]['tokens']
    train_sentence = []
    for i in range(0,len(a)):
        if 'label' in a[i]: # only add element if this sample sentence has been labelled 
            token_element = (a[i]['text'],a[i]['label'])
            train_sentence.append(token_element)
    train_sents.append(train_sentence)

train_sents

[[('DORNBIRN', 'B-ORT'),
  ('In', 'O'),
  ('der', 'O'),
  ('Schulgasse', 'B-STRASSE'),
  ('in', 'O'),
  ('Dornbirn', 'B-ORT'),
  ('hat', 'O'),
  ('eine', 'O'),
  ('71,93', 'B-FLAECHE'),
  ('Quadratmeter', 'O'),
  ('große', 'O'),
  ('Wohnung', 'B-IMMO_TYP'),
  ('für', 'O'),
  ('einen', 'O'),
  ('Quadratmeterpreis', 'O'),
  ('von', 'O'),
  ('5533,71', 'B-GESAMTPREIS'),
  ('Euro', 'O'),
  ('den', 'O'),
  ('Besitzer', 'O'),
  ('gewechselt', 'O'),
  ('.', 'O'),
  ('Dieser', 'O'),
  ('beinhaltet', 'O'),
  ('auch', 'O'),
  ('einen', 'O'),
  ('Pkw-Abstellplatz', 'O'),
  ('.', 'O'),
  ('Käufer', 'O'),
  ('der', 'O'),
  ('Wohnung', 'O'),
  ('mit', 'O'),
  ('9,86', 'B-TERRASSENGROESSE'),
  ('Quadratmetern', 'O'),
  ('Terrasse', 'O'),
  ('ist', 'O'),
  ('die', 'O'),
  ('ValLiLean', 'B-KAEUFER'),
  ('Beteiligungs-', 'I-KAEUFER'),
  ('und', 'I-KAEUFER'),
  ('Immobilienverwaltungs', 'I-KAEUFER'),
  ('GmbH.', 'I-KAEUFER'),
  ('Beim', 'O'),
  ('Verkäufer', 'O'),
  ('handelt', 'O'),
  ('es', 'O'),
  ('s

In [7]:
def word2features(sent, i):
    word = sent[i][0]
    #postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        #'postag': postag, # don't have PoS data
        #'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        #postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            #'-1:postag': postag1,
            #'-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        #postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            #'+1:postag': postag1,
            #'+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
   # return [label for token, postag, label in sent]
    return [label for token, label in sent]

def sent2tokens(sent):
   # return [token for token, postag, label in sent]
    return [token for token, label in sent]

In [8]:
sent2features(train_sents[0])[0]

{'bias': 1.0,
 'word.lower()': 'dornbirn',
 'word[-3:]': 'IRN',
 'word[-2:]': 'RN',
 'word.isupper()': True,
 'word.istitle()': False,
 'word.isdigit()': False,
 'BOS': True,
 '+1:word.lower()': 'in',
 '+1:word.istitle()': True,
 '+1:word.isupper()': False}

In [9]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]
print(y_train[0])
print(len(X_train))
print(len(y_train))

['B-ORT', 'O', 'O', 'B-STRASSE', 'O', 'B-ORT', 'O', 'O', 'B-FLAECHE', 'O', 'O', 'B-IMMO_TYP', 'O', 'O', 'O', 'O', 'B-GESAMTPREIS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-TERRASSENGROESSE', 'O', 'O', 'O', 'O', 'B-KAEUFER', 'I-KAEUFER', 'I-KAEUFER', 'I-KAEUFER', 'I-KAEUFER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-VERKAEUFER', 'I-VERKAEUFER', 'I-VERKAEUFER', 'O', 'O', 'O', 'O', 'O', 'B-GESAMTPREIS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DATUM_VERTRAG', 'I-DATUM_VERTRAG', 'I-DATUM_VERTRAG', 'O', 'O', 'O', 'O', 'B-DATUM_VERBUECHERUNG', 'I-DATUM_VERBUECHERUNG']
140
140


In [10]:
%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train[:120], y_train[:120])

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 3.81 µs


/Users/matthias/opt/anaconda3/envs/nlu/lib/python3.8/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [11]:
eli5.show_weights(crf, top=30)

From \ To,O,B-DATUM_VERBUECHERUNG,I-DATUM_VERBUECHERUNG,B-DATUM_VERTRAG,I-DATUM_VERTRAG,B-FLAECHE,B-GESAMTPREIS,I-GESAMTPREIS,B-IMMO_TYP,I-IMMO_TYP,B-KAEUFER,I-KAEUFER,B-ORT,I-ORT,B-QMPREIS,B-STRASSE,I-STRASSE,B-TERRASSENGROESSE,B-VERKAEUFER,I-VERKAEUFER
O,2.552,1.723,-1.633,1.854,-1.971,2.067,1.294,-1.043,2.03,-3.055,1.855,-2.134,0.941,-1.208,1.395,2.04,-2.133,0.79,1.887,-2.342
B-DATUM_VERBUECHERUNG,-1.413,-0.561,4.746,0.0,0.0,0.0,0.0,0.0,0.0,-0.052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-DATUM_VERBUECHERUNG,-0.576,0.0,1.146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-DATUM_VERTRAG,-0.859,0.0,0.0,-0.525,4.573,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-DATUM_VERTRAG,0.208,0.0,0.0,0.0,4.058,0.0,0.0,0.0,0.0,0.0,0.0,-0.046,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.172
B-FLAECHE,1.616,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,-0.001,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-GESAMTPREIS,1.376,0.0,-0.0,0.0,0.0,0.0,0.0,2.235,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-GESAMTPREIS,0.484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-IMMO_TYP,0.381,0.0,-0.247,0.0,-0.094,0.0,0.0,0.0,-0.163,3.224,0.0,-0.332,0.0,0.0,0.0,0.296,0.0,0.0,0.0,-0.341
I-IMMO_TYP,0.133,0.0,0.0,0.0,0.0,-0.091,0.0,0.0,0.0,4.096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.056,0.0,0.0


In [12]:
labels = list(crf.classes_)
labels.remove('O')
labels

['B-ORT',
 'B-STRASSE',
 'B-FLAECHE',
 'B-IMMO_TYP',
 'B-GESAMTPREIS',
 'B-TERRASSENGROESSE',
 'B-KAEUFER',
 'I-KAEUFER',
 'B-VERKAEUFER',
 'I-VERKAEUFER',
 'B-DATUM_VERTRAG',
 'I-DATUM_VERTRAG',
 'B-DATUM_VERBUECHERUNG',
 'I-DATUM_VERBUECHERUNG',
 'B-QMPREIS',
 'I-IMMO_TYP',
 'I-STRASSE',
 'I-GESAMTPREIS',
 'I-ORT']

In [13]:
X_test = X_train[120:]
y_test = y_train[120:]

In [14]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels)

/Users/matthias/opt/anaconda3/envs/nlu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


0.9002400202334798

In [15]:
# group B and I results
sorted_labels = sorted(labels, key=lambda name: (name[1:], name[0]))
print(metrics.flat_classification_report(y_test, y_pred, labels=sorted_labels, digits=3))

                       precision    recall  f1-score   support

B-DATUM_VERBUECHERUNG      1.000     1.000     1.000        15
I-DATUM_VERBUECHERUNG      1.000     1.000     1.000        15
      B-DATUM_VERTRAG      0.933     0.933     0.933        15
      I-DATUM_VERTRAG      0.931     0.964     0.947        28
            B-FLAECHE      0.944     1.000     0.971        17
        B-GESAMTPREIS      1.000     1.000     1.000        13
        I-GESAMTPREIS      0.000     0.000     0.000         0
           B-IMMO_TYP      0.895     0.773     0.829        22
           I-IMMO_TYP      0.000     0.000     0.000         1
            B-KAEUFER      0.824     0.933     0.875        15
            I-KAEUFER      0.778     1.000     0.875        21
                B-ORT      0.861     0.912     0.886        34
                I-ORT      0.000     0.000     0.000         1
            B-QMPREIS      1.000     1.000     1.000        11
            B-STRASSE      0.917     0.786     0.846  

/Users/matthias/opt/anaconda3/envs/nlu/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass labels=['B-DATUM_VERBUECHERUNG', 'I-DATUM_VERBUECHERUNG', 'B-DATUM_VERTRAG', 'I-DATUM_VERTRAG', 'B-FLAECHE', 'B-GESAMTPREIS', 'I-GESAMTPREIS', 'B-IMMO_TYP', 'I-IMMO_TYP', 'B-KAEUFER', 'I-KAEUFER', 'B-ORT', 'I-ORT', 'B-QMPREIS', 'B-STRASSE', 'I-STRASSE', 'B-TERRASSENGROESSE', 'B-VERKAEUFER', 'I-VERKAEUFER'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/matthias/opt/anaconda3/envs/nlu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matthias/opt/anaconda3/envs/nlu/lib/python3.8/site-packages/sklearn/